In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('paper_data.csv')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import GRU, Dropout, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df[['PV generation','day_length','sunrise_seconds',
       'sunset_seconds','hour_cos','hour_sin','solar_angle']])

scaled_df = pd.DataFrame(scaled_df, columns=['PV generation','day_length','sunrise_seconds',
       'sunset_seconds','hour_cos','hour_sin','solar_angle'])


def create_sequences(df, seq_length):
    sequences = []
    labels = []
    for i in range(len(df) - seq_length):
        seq = df.iloc[i:i+seq_length].values
        label = df['PV generation'].iloc[i+seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

SEQ_LENGTH = 24
sequences, labels = create_sequences(scaled_df, SEQ_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, shuffle=False)


model = Sequential([
    GRU(100, return_sequences=True, input_shape=(SEQ_LENGTH, X_train.shape[2])),
    Dropout(0.2),
    GRU(100),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)


history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])


predictions_test = model.predict(X_test)
predictions_train = model.predict(X_train)


r2_test = r2_score(y_test, predictions_test)
mae_test = mean_absolute_error(y_test, predictions_test)
mse_test = mean_squared_error(y_test, predictions_test)
rmse_test = np.sqrt(mse_test)


r2_train = r2_score(y_train, predictions_train)
mae_train = mean_absolute_error(y_train, predictions_train)
mse_train = mean_squared_error(y_train, predictions_train)
rmse_train = np.sqrt(mse_train)


print(f'--- Test Data ---')
print(f'R² Score: {r2_test}')
print(f'Mean Absolute Error (MAE): {mae_test}')
print(f'Mean Squared Error (MSE): {mse_test}')
print(f'Root Mean Squared Error (RMSE): {rmse_test}')


print(f'--- Train Data ---')
print(f'R² Score: {r2_train}')
print(f'Mean Absolute Error (MAE): {mae_train}')
print(f'Mean Squared Error (MSE): {mse_train}')
print(f'Root Mean Squared Error (RMSE): {rmse_train}')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
238/238 ━━━━━━━━━━━━━━━━━━━━ 20s 56ms/step - loss: 0.0240 - val_loss: 0.0052 - learning_rate: 0.0010
Epoch 2/30
238/238 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - loss: 0.0084 - val_loss: 0.0055 - learning_rate: 0.0010
Epoch 3/30
238/238 ━━━━━━━━━━━━━━━━━━━━ 13s 54ms/step - loss: 0.0069 - val_loss: 0.0043 - learning_rate: 0.0010
Epoch 4/30
238/238 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - loss: 0.0070 - val_loss: 0.0038 - learning_rate: 0.0010
Epoch 5/30
238/238 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - loss: 0.0060 - val_loss: 0.0041 - learning_rate: 0.0010
Epoch 6/30
238/238 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - loss: 0.0062 - val_loss: 0.0041 - learning_rate: 0.0010
Epoch 7/30
238/238 ━━━━━━━━━━━━━━━━━━━━ 22s 59ms/step - loss: 0.0061 - val_loss: 0.0048 - learning_rate: 0.0010
Epoch 8/30
238/238 ━━━━━━━━━━━━━━━━━━━━ 19s 54ms/step - loss: 0.0058 - val_loss: 0.0039 - learning_rate: 0.0010
Epoch 9/30
238/238 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - loss: 0.0058 - val_loss: 0.0037 - learning_rate: